In [1]:
import imaplib, email, random, re
from helper import SpamDetection, sort_email

In [2]:
host = 'imap.gmail.com'

with open('/users/gursi/desktop/gmail_login.txt', 'r') as f : 
    username = f.readline().replace('\n','')
    password = f.readline().replace('\n','')

imap = imaplib.IMAP4_SSL(host)
imap.login(username, password)
print('Logged in.')

Logged in.


In [3]:
detect = SpamDetection()
spam_folder_name = "\"" + "Collegeboard spam" + "\""

# mail.select(spam_folder_name, readonly=False)
imap.select("inbox", readonly=False)

_, mails = imap.search(None, 'UNSEEN')
mails = mails[0].split()

if len(mails) == 0 : 
    print('No unseen messages')
else : 
    for i,uid in zip([1],mails):
        _, data = imap.fetch(uid, '(RFC822)')
        _, b = data[0]
        email_msg = email.message_from_bytes(b)
        print(f'uid: {uid}')
        for header in ['subject', 'to', 'from', 'date']:
            print(f'{header}: {email_msg[header]}')
        for part in email_msg.walk():
            if part.get_content_type()=='text/plain' :
                body = part.get_payload(decode=True).decode()
                spam = detect.classify(body, email_msg['from'], email_msg['subject'], print_body=True)
                sort_email(imap, spam, uid, spam_folder_name)

uid: b'3980'
subject: Sample email 
to: "gursimar.batra@gmail.com" <gursimar.batra@gmail.com>
from: Gursimar Singh <gursimarsingh.stu@npsinternational.edu.sg>
date: Sat, 25 Sep 2021 11:51:26 +0000
Body: This is a sample email

Spam detected.
Mail marked as unseen.
Mail relabeled.
Mail deleted.
